In [31]:
import pandas as pd
from sklearn.impute import SimpleImputer
from os import lseek
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [32]:
df = pd.read_csv('/content/horse.csv')
df

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,NaN,more_3_sec,...,45.0,8.4,NaN,NaN,died,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,NaN,NaN,pale_cyanotic,less_3_sec,...,50.0,85.0,cloudy,2.0,euthanized,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,33.0,6.7,NaN,NaN,lived,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,48.0,7.2,serosanguious,5.3,died,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,NaN,NaN,dark_cyanotic,more_3_sec,...,74.0,7.4,NaN,NaN,died,no,4300,0,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,yes,adult,533886,NaN,120.0,70.0,cold,NaN,pale_cyanotic,more_3_sec,...,55.0,65.0,NaN,NaN,euthanized,no,3205,0,0,no
295,no,adult,527702,37.2,72.0,24.0,cool,increased,pale_cyanotic,more_3_sec,...,44.0,NaN,serosanguious,3.3,euthanized,yes,2208,0,0,yes
296,yes,adult,529386,37.5,72.0,30.0,cold,reduced,pale_cyanotic,less_3_sec,...,60.0,6.8,NaN,NaN,died,yes,3205,0,0,no
297,yes,adult,530612,36.5,100.0,24.0,cool,reduced,pale_pink,less_3_sec,...,50.0,6.0,serosanguious,3.4,lived,yes,2208,0,0,yes


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                299 non-null    object 
 1   age                    299 non-null    object 
 2   hospital_number        299 non-null    int64  
 3   rectal_temp            239 non-null    float64
 4   pulse                  275 non-null    float64
 5   respiratory_rate       241 non-null    float64
 6   temp_of_extremities    243 non-null    object 
 7   peripheral_pulse       230 non-null    object 
 8   mucous_membrane        252 non-null    object 
 9   capillary_refill_time  267 non-null    object 
 10  pain                   244 non-null    object 
 11  peristalsis            255 non-null    object 
 12  abdominal_distention   243 non-null    object 
 13  nasogastric_tube       195 non-null    object 
 14  nasogastric_reflux     193 non-null    object 
 15  nasoga

In [34]:
#limpieza
dfl = df[{'rectal_temp','pulse','respiratory_rate','nasogastric_reflux_ph','packed_cell_volume','total_protein','abdomo_protein'}]
dfl

<ipython-input-34-0287e8f09b07>:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dfl = df[{'rectal_temp','pulse','respiratory_rate','nasogastric_reflux_ph','packed_cell_volume','total_protein','abdomo_protein'}]


,abdomo_protein,nasogastric_reflux_ph,rectal_temp,total_protein,packed_cell_volume,respiratory_rate,pulse
0,NaN,NaN,38.5,8.4,45.0,28.0,66.0
1,2.0,NaN,39.2,85.0,50.0,20.0,88.0
2,NaN,NaN,38.3,6.7,33.0,24.0,40.0
3,5.3,5.0,39.1,7.2,48.0,84.0,164.0
4,NaN,NaN,37.3,7.4,74.0,35.0,104.0
...,...,...,...,...,...,...,...
294,NaN,NaN,NaN,65.0,55.0,70.0,120.0
295,3.3,NaN,37.2,NaN,44.0,24.0,72.0
296,NaN,NaN,37.5,6.8,60.0,30.0,72.0
297,3.4,NaN,36.5,6.0,50.0,24.0,100.0


In [35]:
#Imputacion
imputer = SimpleImputer(strategy='median')
#para utilizar mediana o asi se pone imputer = SimpleImputer(strategy='median)
imputer.fit(dfl)
#fit calcula la   estadistica para imputar (no hace la imputacion solo los calculos)
imputer.statistics_
#Trae la media de cada columna del df
dfli = imputer.transform(dfl)
dfii = pd.DataFrame(dfli)
dfii = dfii.rename(columns={dfii.columns[0]: 'abdomo_protein', dfii.columns[1]: 'nasogastric_reflux_ph', dfii.columns[2]: 'rectal_temp', dfii.columns[3]: 'total_protein', dfii.columns[4]: 'packed_cell_volume', dfii.columns[5]: 'respiratory_rate', dfii.columns[6]: 'pulse'})
dfii

,abdomo_protein,nasogastric_reflux_ph,rectal_temp,total_protein,packed_cell_volume,respiratory_rate,pulse
0,2.3,5.0,38.5,8.4,45.0,28.0,66.0
1,2.0,5.0,39.2,85.0,50.0,20.0,88.0
2,2.3,5.0,38.3,6.7,33.0,24.0,40.0
3,5.3,5.0,39.1,7.2,48.0,84.0,164.0
4,2.3,5.0,37.3,7.4,74.0,35.0,104.0
...,...,...,...,...,...,...,...
294,2.3,5.0,38.2,65.0,55.0,70.0,120.0
295,3.3,5.0,37.2,7.5,44.0,24.0,72.0
296,2.3,5.0,37.5,6.8,60.0,30.0,72.0
297,3.4,5.0,36.5,6.0,50.0,24.0,100.0


In [41]:
objeto = MinMaxScaler()
objeto.fit(dfii)
dfiii = objeto.transform(dfii)
dfiv = pd.DataFrame(dfiii)
dfv = dfiv.rename(columns={dfiv.columns[0]: 'abdomo_protein', dfiv.columns[1]: 'nasogastric_reflux_ph', dfiv.columns[2]: 'rectal_temp', dfiv.columns[3]: 'total_protein', dfiv.columns[4]: 'packed_cell_volume', dfiv.columns[5]: 'respiratory_rate', dfiv.columns[6]: 'pulse'})
dfv

,abdomo_protein,nasogastric_reflux_ph,rectal_temp,total_protein,packed_cell_volume,respiratory_rate,pulse
0,0.22,0.615385,0.574074,0.059510,0.423077,0.227273,0.233766
1,0.19,0.615385,0.703704,0.953326,0.519231,0.136364,0.376623
2,0.22,0.615385,0.537037,0.039673,0.192308,0.181818,0.064935
3,0.52,0.615385,0.685185,0.045508,0.480769,0.863636,0.870130
4,0.22,0.615385,0.351852,0.047841,0.980769,0.306818,0.480519
...,...,...,...,...,...,...,...
294,0.22,0.615385,0.518519,0.719953,0.615385,0.704545,0.584416
295,0.32,0.615385,0.333333,0.049008,0.403846,0.181818,0.272727
296,0.22,0.615385,0.388889,0.040840,0.711538,0.250000,0.272727
297,0.33,0.615385,0.203704,0.031505,0.519231,0.181818,0.454545
